In [1]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import numpy

from collections import Counter

pathPrefix = '/Users/friedman/Desktop/mnt'

sys.path.append(pathPrefix + '/ifs/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import clonality_analysis_util
import re


In [7]:
allImpactMutsUnfiltered = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/mskImpactAsOfMarch2019/dmp/mskimpact/data_mutations_unfiltered.txt', skiprows=[0])


/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (17,18,19,45,46,48,50,51,52,53,54,55,56,57,60,61,62,63,64,65,74,75,76,77,78,80,81,85,86,87,89,90,91,92,93,98,100,101,102,103,104,105,106,107,108,110,111,112,113,114,115,116,117,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
endometrialHyperIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Endometrial Cancer', hypermutantStatus = 'Hypermutated')
colorectalHyperIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Colorectal Cancer', hypermutantStatus = 'Hypermutated')
gliomaHyperIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Glioma', hypermutantStatus = 'Hypermutated')

endometrialNormalIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Endometrial Cancer', hypermutantStatus = 'Normal')
colorectalNormalIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Colorectal Cancer', hypermutantStatus = 'Normal')
gliomaNormalIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Glioma', hypermutantStatus = 'Normal')

In [11]:
endometrialHyperMuts = allImpactMutsUnfiltered[allImpactMutsUnfiltered['Tumor_Sample_Barcode'].isin(endometrialHyperIds)]
colorectalHyperMuts = allImpactMutsUnfiltered[allImpactMutsUnfiltered['Tumor_Sample_Barcode'].isin(colorectalHyperIds)]
gliomaHyperMuts = allImpactMutsUnfiltered[allImpactMutsUnfiltered['Tumor_Sample_Barcode'].isin(gliomaHyperIds)]

endometrialNormalMuts = allImpactMutsUnfiltered[allImpactMutsUnfiltered['Tumor_Sample_Barcode'].isin(endometrialNormalIds)]
colorectalNormalMuts = allImpactMutsUnfiltered[allImpactMutsUnfiltered['Tumor_Sample_Barcode'].isin(colorectalNormalIds)]
gliomaNormalMuts = allImpactMutsUnfiltered[allImpactMutsUnfiltered['Tumor_Sample_Barcode'].isin(gliomaNormalIds)]

In [11]:
impactSigs = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/signatures_from_unfiltered_maf.txt')

impactSigs = mutationSigUtils.merge_signature_columns(impactSigs)
impactSigs['dominantSignature'] = impactSigs.apply(lambda row: 
        mutationSigUtils.get_dominant_signature(row.to_dict(), cols=None, prefix='mean', notEnoughMuts= True), axis=1)

dominantSignaturesDict = dict(zip(impactSigs['Tumor_Sample_Barcode'], impactSigs['dominantSignature']))

In [33]:
poleCases = set(impactSigs[(impactSigs['dominantSignature'] == 'mean_10') & (impactSigs['Nmut_Mb'] > 30)]['Tumor_Sample_Barcode'])
mmrCases = set(impactSigs[(impactSigs['dominantSignature'] == 'mean_MMR') & (impactSigs['Nmut_Mb'] > 30)]['Tumor_Sample_Barcode'])
tmzCases = set(impactSigs[(impactSigs['dominantSignature'] == 'mean_11') & (impactSigs['Nmut_Mb'] > 30)]['Tumor_Sample_Barcode'])


In [34]:
poleMuts = endometrialMuts[endometrialMuts['Tumor_Sample_Barcode'].isin(poleCases)]
mmrMuts = endometrialMuts[endometrialMuts['Tumor_Sample_Barcode'].isin(mmrCases)]
tmzMuts = endometrialMuts[endometrialMuts['Tumor_Sample_Barcode'].isin(tmzCases)]

In [41]:
endometrialNormalMuts.to_csv('/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myAdjustedDataFiles/endometrialNormalMutationsUnfiltered.maf', index=False, sep='\t')
colorectalNormalMuts.to_csv('/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myAdjustedDataFiles/colorectalNormalMutationsUnfiltered.maf', index=False, sep='\t')
gliomaNormalMuts.to_csv('/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myAdjustedDataFiles/gliomaNormalMutationsUnfiltered.maf', index=False, sep='\t')

endometrialHyperMuts.to_csv('/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myAdjustedDataFiles/endometrialHypermutatedMutationsUnfiltered.maf', index=False, sep='\t')
colorectalHyperMuts.to_csv('/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myAdjustedDataFiles/colorectalHypermutatedMutationsUnfiltered.maf', index=False, sep='\t')
gliomaHyperMuts.to_csv('/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myAdjustedDataFiles/gliomaHypermutatedMutationsUnfiltered.maf', index=False, sep='\t')

poleMuts.to_csv('/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myAdjustedDataFiles/poleMutationsUnfiltered.maf', index=False, sep='\t')
mmrMuts.to_csv('/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myAdjustedDataFiles/mmrMutationsUnfiltered.maf', index=False, sep='\t')
tmzMuts.to_csv('/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myAdjustedDataFiles/tmzMutationsUnfiltered.maf', index=False, sep='\t')

**QC of DNDSCV outputs**<br><br><br><br>

In [47]:
colorectalDndsCv = pd.read_table('/Users/friedman/Desktop/hypermutationAnalysisProj/colorectalCancerHyperDNDS.tsv')
endometrialDndsCv = pd.read_table('/Users/friedman/Desktop/hypermutationAnalysisProj/endometrialCancerHyperDNDS.tsv')
gliomaDndsCv = pd.read_table('/Users/friedman/Desktop/hypermutationAnalysisProj/gliomaHyperDNDS.tsv')

In [21]:
def qc_dnds_cv(dndsResults, maf):
    indelGeneDict = dict(zip(dndsResults['gene_name'], dndsResults['n_ind']))
    
    for gene in set(dndsResults['gene_name']):
        geneMaf = maf[maf['Hugo_Symbol'] == gene]
        indelMaf = geneMaf[(geneMaf['Variant_Type'] == 'INS') | (geneMaf['Variant_Type'] == 'DEL')]
        #nIndels = indelMaf.shape[0]
        nIndels = len(set(indelMaf['HGVSp_Short']))

        nIndelsDnds = indelGeneDict[gene]
        print gene, nIndels - nIndelsDnds

In [54]:
colorectalDndsCv['chrPosGene'] = colorectalDndsCv.apply(lambda row: str(row['chr']) + '_' + str(row['pos']) + '_' + row['gene'], axis=1)

colorectalHyperMuts['chrPosGene'] = colorectalHyperMuts.apply(lambda row: str(row['Chromosome']) + '_' + str(row['Start_Position']) + '_' + str(row['Hugo_Symbol']), axis=1)



/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [89]:
def quantify_dnds_fuckup(gene, posDict, classDict):
    DNDS = set(colorectalDndsCv[colorectalDndsCv['gene'] == gene]['chrPosGene'])
    DNDScount = Counter(colorectalDndsCv[colorectalDndsCv['gene'] == gene]['chrPosGene'])
    Obs = set(colorectalHyperMuts[colorectalHyperMuts['Hugo_Symbol'] == gene]['chrPosGene'])
    Obscount = Counter(colorectalHyperMuts[colorectalHyperMuts['Hugo_Symbol'] == gene]['chrPosGene'])
    missingVals = Obs - DNDS
    absMissing = Obscount - DNDScount
    
    #print absMissing
    
    x = []
    counts = []
    for val in missingVals:
        if val in posDict:
            x.append(classDict[val])
            counts.append((posDict[val], absMissing[val]))
    
    print Counter(counts)
    print ''
    print Counter(x)

In [71]:
posDict = dict(zip(colorectalHyperMuts['chrPosGene'], colorectalHyperMuts['HGVSp_Short']))
varClassDict = dict(zip(colorectalHyperMuts['chrPosGene'], colorectalHyperMuts['Variant_Classification']))

In [10]:
dfKras = pd.read_table('/Users/friedman/Desktop/hypermutationAnalysisProj/krasDNDS_test.tsv')

In [12]:
#for debug work with martincorena
def check_inclusion(x):
    if isinstance(x, basestring):
        if 'p.' in x: return True
        else: return False
    else:
        return False
        
dfKras['inclusion'] = dfKras['hgvs'].apply(lambda x: check_inclusion(x))
dfKrasLim = dfKras[dfKras['inclusion'] == True]
dfKrasLim.to_csv('~/Desktop/dndsCVBugTest.tsv', index=False, sep='\t')

In [13]:
dfKrasLim

,sampleID,chr,pos,ref,alt,gene,hgvs,inclusion
1,P-0039054-T01-IM6,12,25380283,C,T,KRAS,p.A59T,True
3,P-0038520-T01-IM6,12,25398281,C,T,KRAS,p.G13D,True
5,P-0004379-T02-IM6,12,25368397,A,T,KRAS,p.I183N,True
6,P-0037690-T01-IM6,12,25380283,C,T,KRAS,p.A59T,True
7,P-0033467-T01-IM6,12,25398282,C,A,KRAS,p.G13C,True
8,P-0033682-T01-IM6,12,25380275,T,G,KRAS,p.Q61H,True
9,P-0033682-T01-IM6,12,25398279,C,T,KRAS,p.V14I,True
11,P-0035146-T01-IM6,12,25380283,C,T,KRAS,p.A59T,True
13,P-0033583-T01-IM6,12,25398281,C,T,KRAS,p.G13D,True
14,P-0031059-T01-IM6,12,25398284,C,A,KRAS,p.G12V,True
